In [1]:
import numpy as np
import astropy.io.fits as pyfits
import matplotlib.pyplot as plt
import glob

#this is for Sven Buder's fits file reader function which reads fits into a Python dict of numpy arrays

In [2]:
from spectres import spectres
#these are for running spectres which allows us to resample spectra and their errors with very little effort 

In [3]:
#sobject_ids = [161217002601184,170418003701205] 
#sobject_ids = [170418003701205] 
import pandas as pd 

df_full= pd.read_csv("/disks/nut/home/46670564/praveen/code/galah-test-nut/GALAH_all_sobject_ids.csv") #full GALAH catalog

df_missing = pd.read_csv("/disks/nut/home/46670564/praveen/code/galah-test-nut/GALAH_DR3_list_missing_normalized_spectra_v2.csv", header=None) #GALAH objects with missing normalised spectra 
df_full.drop(['Unnamed: 0'], axis=1)
df_missing.columns = ['sobject_id']

df_temp = pd.merge(df_full, df_missing, how='outer', indicator=True)
df_final_sobject_ids_temp = df_temp.loc[df_temp['_merge'] == "left_only"].drop(["Unnamed: 0","_merge"], axis=1)

np.random.seed(1) #set seed here 
sobject_ids = df_final_sobject_ids_temp.loc[np.random.choice(df_final_sobject_ids_temp.index, size=5000)]["sobject_id"].to_numpy().tolist()

#input the object IDs, change this later to handle a list of 30K object IDs
#these object IDs will drive the read_spectra method
#calling read_spectra using object IDs in a loop will be useful to iterate through all the objects and generate dict() objects for their spectra 

In [4]:
GRID_SIZE = 0.06
LOWER_LAMBDA = 6472.5
UPPER_LAMBDA = 6740

#the grid size was determined by looking at camera 3 for a few fits files and checking the step between lambda values that are general enough for all fits files 
#lower and upper limites are from Sven Buder's example for camera 3
#this constants will have to be adjusted depending on the element being studied and the minimum resolution of the wave grid (lambda values i.e. lambda[i+1] - lambda[i])

In [5]:
regrid = np.arange(LOWER_LAMBDA, UPPER_LAMBDA, GRID_SIZE) 

#this is the grid onto which spectres will resampple 

In [6]:
def read_spectra(sobject_id):
    fits_files = [[],[],[],[]]
    for each_ccd in [1,2,3,4]:
        fits_files[each_ccd-1] = glob.glob('/data/praveen/galah-total/galah/dr3/spectra/hermes/'+str(sobject_id)+str(each_ccd)+'.fits') #this is reading fits files from file and not downloading directly 

    spectrum = dict()
    for each_ccd in [1,2,3,4]: #GALAH uses indexing from 1 - 4
        if fits_files[each_ccd-1]!=[]: #just using zero indexing here 
            fits = pyfits.open(fits_files[each_ccd-1][0]) 
            
            # Extension 0: Reduced spectrum
            # Extension 1: Relative error spectrum
            # Extension 4: Normalised spectrum, NB: cut for CCD4

            # Extract wavelength grid for the reduced spectrum
            start_wavelength = fits[0].header["CRVAL1"]
            dispersion       = fits[0].header["CDELT1"]
            nr_pixels        = fits[0].header["NAXIS1"]
            reference_pixel  = fits[0].header["CRPIX1"]

            if reference_pixel == 0:
                reference_pixel = 1
            spectrum['wave_red_'+str(each_ccd)] = ((np.arange(0,nr_pixels)--reference_pixel+1)*dispersion+start_wavelength) #this is the reduced spectrum 

            # Extract wavelength grid for the normalised spectrum
            start_wavelength = fits[4].header["CRVAL1"]
            dispersion       = fits[4].header["CDELT1"]
            nr_pixels        = fits[4].header["NAXIS1"]
            reference_pixel  = fits[4].header["CRPIX1"]

            if reference_pixel == 0:
                reference_pixel=1
            spectrum['wave_norm_'+str(each_ccd)] = ((np.arange(0,nr_pixels)--reference_pixel+1)*dispersion+start_wavelength) #this is the normalised spectrum 

            spectrum['sob_red_'+str(each_ccd)]  = np.array(fits[0].data)
            spectrum['uob_red_'+str(each_ccd)]  = np.array(fits[0].data * fits[1].data)

            spectrum['sob_norm_'+str(each_ccd)] = np.array(fits[4].data)
            if each_ccd != 4:
                spectrum['uob_norm_'+str(each_ccd)] = np.array(fits[4].data * fits[1].data)
            else:
                # for normalised error of CCD4, only used appropriate parts of error spectrum
                spectrum['uob_norm_4'] = np.array(fits[4].data * (fits[1].data)[-len(spectrum['sob_norm_4']):])

            fits.close()
        else:
            spectrum['wave_red_'+str(each_ccd)] = []
            spectrum['wave_norm_'+str(each_ccd)] = []
            spectrum['sob_red_'+str(each_ccd)] = []
            spectrum['sob_norm_'+str(each_ccd)] = []
            spectrum['uob_red_'+str(each_ccd)] = []
            spectrum['uob_norm_'+str(each_ccd)] = []
    
    spectrum['wave_red'] = np.concatenate(([spectrum['wave_red_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['wave_norm'] = np.concatenate(([spectrum['wave_norm_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['sob_red'] = np.concatenate(([spectrum['sob_red_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['sob_norm'] = np.concatenate(([spectrum['sob_norm_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['uob_red'] = np.concatenate(([spectrum['uob_red_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))
    spectrum['uob_norm'] = np.concatenate(([spectrum['uob_norm_'+str(each_ccd)] for each_ccd in [1,2,3,4]]))

    return spectrum 
    
    #returns a 30 "row" dict of numpy array per row 'wave_red_x' is the key for the key value pair
    #camera 3 will be the more useful canmera for Li spectra 

In [7]:
def resample_spectra(spectrum, camera, verbose):

    spec_resample, spec_errs_resample = spectres(regrid,  spectrum['wave_norm_'+str(camera)], spectrum['sob_norm_'+str(camera)], spec_errs= spectrum['uob_norm_'+str(camera)],verbose=verbose) 

    return spec_resample, spec_errs_resample

#for Li pick camera 3

In [8]:
#open an empty dict to store the results 
resampled_spectra_collection = {}
resampled_spectra_collection['spec_resample'] = []

resampled_error_collection = {}
resampled_error_collection['error_resample'] = []

In [9]:
for sobject_id in sobject_ids:
    #this is the normalised resampled spectra 
    temp_spectrum = resample_spectra(read_spectra(sobject_id), 3, False)
    temp_spectrum[0][np.isnan(temp_spectrum[0])] = 1 #padding 
    resampled_spectra_collection['spec_resample'].append(temp_spectrum[0])

    #these are the error spectra
    #calculate mean error for padding 

    non_na_values = temp_spectrum[1][~np.isnan(temp_spectrum[1])]
    mean_error = np.mean(non_na_values)

    temp_spectrum[1][np.isnan(temp_spectrum[1])] = mean_error
    resampled_error_collection['error_resample'].append(temp_spectrum[1])

In [10]:
import h5py

#save the resampled spectra to be used as inputs to the training set
hf_spec = h5py.File("/data/praveen/results/resampled_test_spectra1.h5", "w")
hf_spec.create_dataset('spectra', data=resampled_spectra_collection['spec_resample'])
hf_spec.close()

In [ ]:
#save the wavelength grid 
hf_grid = h5py.File("/data/praveen/results/wl_grid.h5", "w")
hf_grid.create_dataset('wl_grid', data=regrid)
hf_grid.close()

In [ ]:
#save the error spectra
hf_error = h5py.File("/data/praveen/results/resampled_test_errors1.h5", "w")
hf_error.create_dataset('errors', data=resampled_error_collection['error_resample'])
hf_error.close()